In [ ]:
from pathlib import Path
import glob

import corner
import emcee
import yaml
import numpy as np
%pylab inline

import constants

In [ ]:
IVOL_DENS_MAP = {
    33: -0.16804129,
    55: 0.007637806,
    17: 0.15005589,
    59: 0.05978356
}

# survey: (burnin, thin)
MCMC_PARAM_MAP = {
    'clamato': (68, 16),
    'zDeep': (68, 16)
}

MCMC_BASE = Path(constants.XCORR_DIR_BASE) / 'mock' / 'mcmc'

In [ ]:
bias_mcmc_dict = {s: {} for s in MCMC_PARAM_MAP.keys()}

for survey, (burnin, thin) in MCMC_PARAM_MAP.items():
    data_dir = MCMC_BASE / survey
    for cfg_path in glob.glob(str(data_dir / f'{survey}*.yaml')):
        with open(cfg_path, 'r') as f:
            ivol = yaml.safe_load(f)['ivol']
        chain_filename = 'chain_' + cfg_path.split('/')[-1][:-5] + '.hdf5'
        chain_path = '/'.join(cfg_path.split('/')[:-1] + [chain_filename])
        backend = emcee.backends.HDFBackend(chain_path, read_only=True)
        samples = backend.get_chain(flat=True, discard=burnin, thin=thin)
        assert samples.size > 0
        assert np.sum(np.isnan(samples)) == 0
        bias_median = np.median(samples[:, 0])
        if ivol not in bias_mcmc_dict[survey]:
            bias_mcmc_dict[survey][ivol] = []
        bias_mcmc_dict[survey][ivol].append(bias_median)

In [ ]:
def plot_dens_vs_bias(survey_name):
    bias_dict = bias_mcmc_dict[survey_name]
    for ivol, bias_list in bias_dict.items():
        mean_dens = IVOL_DENS_MAP[ivol]
        plt.scatter(np.repeat(mean_dens, len(bias_list)), bias_list, alpha=0.5, color='grey')
        plt.scatter(mean_dens, np.mean(bias_list), color='red')

In [ ]:
plot_dens_vs_bias('clamato')
plt.scatter(0.06294089, 3.87, color='blue', marker='*')

In [ ]:
plot_dens_vs_bias('zDeep')
plt.scatter(0.06294089, 3.93, color='blue', marker='*')